In [1]:
import cv2
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image

In [2]:
def extract_frames_from_all_videos(root_dir, frame_dir, sample_rate=5):
    for folder in os.listdir(root_dir):
        folder_path = os.path.join(root_dir, folder)
        if os.path.isdir(folder_path):
            for video_file in os.listdir(folder_path):
                if video_file.endswith('.mp4'):
                    video_path = os.path.join(folder_path, video_file)
                    video_frame_dir = os.path.join(frame_dir, folder)
                    if not os.path.exists(video_frame_dir):
                        os.makedirs(video_frame_dir)
                    extract_frames(video_path, video_frame_dir, sample_rate)


In [3]:

def extract_frames(video_path, frame_dir, sample_rate=5):
    video = cv2.VideoCapture(video_path)
    count = 0
    while True:
        ret, frame = video.read()
        if not ret:
            break
        if count % sample_rate == 0:
            cv2.imwrite(os.path.join(frame_dir, f'frame{count}.jpg'), frame)
        count += 1
    video.release()

extract_frames_from_all_videos(r'C:\Fce Auth\dataset', r'C:\Fce Auth\frames')


In [4]:
def load_data(directory):
    datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

    train_generator = datagen.flow_from_directory(
        directory,
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary',
        subset='training')

    validation_generator = datagen.flow_from_directory(
        directory,
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary',
        subset='validation')

    return train_generator, validation_generator

In [12]:
train_gen, val_gen = load_data(r'C:\Fce Auth\frames')


Found 135 images belonging to 6 classes.
Found 30 images belonging to 6 classes.


In [13]:
def create_model():
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [14]:
def train_model(model, train_gen, val_gen, epochs=10):
    history = model.fit(train_gen, validation_data=val_gen, epochs=epochs)
    return model

In [15]:
def preprocess_frame(frame):
    frame_resized = cv2.resize(frame, (224, 224))
    frame_array = image.img_to_array(frame_resized)
    frame_array = np.expand_dims(frame_array, axis=0)
    frame_array = frame_array / 255.0
    return frame_array

In [16]:
def predict_frame(frame, model):
    processed_frame = preprocess_frame(frame)
    prediction = model.predict(processed_frame)
    return prediction[0][0]

In [17]:
def is_blank_frame(frame, threshold=30):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    mean_intensity = np.mean(gray_frame)
    return mean_intensity < threshold

In [18]:
def webcam_prediction(model):
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Unable to get image")
            continue

        if is_blank_frame(frame):
            cv2.putText(frame, 'Unable to get image', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        else:
            prediction = predict_frame(frame, model)
            label = "Real" if prediction > 0.5 else "Fake"
            confidence = prediction if prediction > 0.5 else 1 - prediction
            label_text = f'{label}: {confidence:.2f}'
            cv2.putText(frame, label_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        cv2.imshow('Webcam Prediction', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [ ]:
if __name__ == "__main__":
    dataset_path = r'C:\Fce Auth\frames'
    train_gen, val_gen = load_data(dataset_path)
    model = create_model()
    trained_model = train_model(model, train_gen, val_gen)
  

Found 135 images belonging to 6 classes.
Found 30 images belonging to 6 classes.
Epoch 1/10


In [ ]:
  webcam_prediction(trained_model)